---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
import numpy as np

In [66]:
#Insert lat and lon in address embbeded in adresses csv 
add = pd.read_csv('addresses.csv') 
lat = pd.read_csv('latlons.csv') 
geoRef = pd.merge(add,lat, how='left', on='address')
geoRef.set_index(['ticket_id'], inplace=True)
geoRef=geoRef[['lat','lon']]

train = pd.read_csv('train.csv', sep=',', encoding='cp1252', low_memory=False,
                    parse_dates = ['ticket_issued_date', 'hearing_date'], index_col='ticket_id')
train = pd.merge(train, geoRef, how='left', on='ticket_id')

test = pd.read_csv('test.csv', sep=',', encoding='cp1252', low_memory=False,
                    parse_dates = ['ticket_issued_date', 'hearing_date'], index_col='ticket_id')
test = pd.merge(test, geoRef, how='left', on='ticket_id')

In [67]:
print(train.compliance.value_counts(dropna=False))
train = train.dropna(subset=['compliance'])

0.0    148283
NaN     90426
1.0     11597
Name: compliance, dtype: int64


In [68]:
print(train.groupby('agency_name').compliance.agg(['count', 'sum','mean']))
train = train[train['agency_name'] != 'Neighborhood City Halls']

                                                count     sum      mean
agency_name                                                            
Buildings, Safety Engineering & Env Department  95863  5823.0  0.060743
Department of Public Works                      52445  4718.0  0.089961
Detroit Police Department                        4464   588.0  0.131720
Health Department                                7107   468.0  0.065851
Neighborhood City Halls                             1     0.0  0.000000


In [69]:
tr = train.disposition.unique().tolist()
te = test.disposition.unique().tolist()
set(tr+te)

disposition_replace = {'Responsible by Default': 'By default',
                       'Responsible by Determination': 'By determination',
                       'Responsible (Fine Waived) by Admis': 'Fine Waived',
                       'Responsible (Fine Waived) by Deter': 'Fine Waived',
                       'Responsible - Compl/Adj by Default': 'By default',
                       'Responsible - Compl/Adj by Determi': 'By determination',
                       'Responsible by Admission': 'By admission',
                       'Responsible by Dismissal': 'By default'}

train.disposition.replace(disposition_replace, inplace=True)
print(train.groupby('disposition').compliance.agg(['count', 'sum']))

test.disposition.replace(disposition_replace, inplace=True)
print(test.groupby('disposition').agency_name.agg(['count']))

                   count     sum
disposition                     
By admission       13701  3768.0
By default        138340  5285.0
By determination    7643  2349.0
Fine Waived          195   195.0
                  count
disposition            
By admission       4484
By default        51609
By determination   4126
Fine Waived         782


In [98]:
print(train.groupby('discount_amount').compliance.agg(['count', 'sum', 'mean']).sort_values('discount_amount'))
print(test.groupby('discount_amount').agency_name.agg(['count']).sort_values('discount_amount'))
train['discount'] = train.discount_amount.apply(lambda x:1 if x > 0 else 0)
test['discount'] = test.discount_amount.apply(lambda x:1 if x > 0 else 0)

                  count      sum      mean
discount_amount                           
0.0              158699  10474.0  0.065999
3.0                  19     19.0  1.000000
5.0                 167    159.0  0.952096
10.0                155    141.0  0.909677
13.0                  1      1.0  1.000000
20.0                135    122.0  0.903704
25.0                605    587.0  0.970248
30.0                 17     17.0  1.000000
40.0                  1      1.0  1.000000
50.0                 43     40.0  0.930233
100.0                16     16.0  1.000000
250.0                 6      5.0  0.833333
350.0                15     15.0  1.000000
                 count
discount_amount       
0.0              60239
3.0                  1
5.0                228
10.0               191
13.0                 9
20.0               146
25.0                94
30.0                 4
40.0                 1
50.0                59
75.0                 4
100.0               22
150.0                2
250.0     

In [111]:
train.groupby('judgment_amount').compliance.agg(['count', 'sum', 'mean', 'std']).sort_index(ascending=True)
cut = [-1, 100, 150, 300, float("inf")]

train['judgment_level'] = pd.cut(train.judgment_amount, bins=cut)
print(train.groupby('judgment_level').compliance.agg(['count', 'sum', 'mean']).sort_index(ascending=True))

test['judgment_level'] = pd.cut(test.judgment_amount, bins=cut)
print(test.groupby('judgment_level').agency_name.agg(['count']).sort_index(ascending=True))

                 count     sum      mean
judgment_level                          
(-1.0, 100.0]    21990  2154.0  0.097954
(100.0, 150.0]   15490  1814.0  0.117108
(150.0, 300.0]   20690  3912.0  0.189077
(300.0, inf]    101709  3717.0  0.036545
                count
judgment_level       
(-1.0, 100.0]   18282
(100.0, 150.0]   8395
(150.0, 300.0]  10850
(300.0, inf]    23474


In [117]:
a = train.groupby('state').compliance.agg(['count', 'sum']).sort_values('count', ascending=False)
a['compl_rate'] = a['sum']/a['count']
a['count_rate'] = a['count']/len(train)
#print(a)

states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME',
          'MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
          'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

train['state_cat'] = train.state.apply(lambda x: x if x in states else 'NOT US')
train.loc[(train.state_cat != 'NOT US') & (train.state_cat != 'MI'),'state_cat'] = 'NOT MI'
train.loc[train.state.isna(),'state_cat'] = 'MI' #Missing values most Frequent
print(train.groupby('state_cat').compliance.agg(['count', 'sum']).sort_values('count', ascending=False))

test['state_cat'] = test.state.apply(lambda x: x if x in states else 'NOT US')
test.loc[(test.state_cat != 'NOT US') & (test.state_cat != 'MI'),'state_cat'] = 'NOT MI'
test.loc[test.state.isna(),'state_cat'] = 'MI' #Missing values most Frequent
print(test.groupby('state_cat').agency_name.agg(['count']).sort_values('count', ascending=False))

            count      sum
state_cat                 
MI         143738  10603.0
NOT MI      15952    973.0
NOT US        189     21.0
           count
state_cat       
MI         52197
NOT MI      8494
NOT US       310


In [150]:
#lat and lon is filled with rhe average of 50 most frequencies locations
mtrain = train.groupby(['lon','lat']).compliance.agg(['count', 'sum', 'mean', 'std']).sort_values('count',ascending=False)
lon = mtrain[mtrain['count']>50].index.get_level_values(0).tolist()
lat = mtrain[mtrain['count']>50].index.get_level_values(1).tolist()
lon = round(sum(lon) / float(len(lon)),6)
lat = round(sum(lat) / float(len(lat)),6)
train.loc[train.lon.isna(),'lon'] = lon #fill in missing values
train.loc[train.lat.isna(),'lat'] = lat #fill in missing values
print(mtrain[mtrain['count']>50])
print('----locations replaced (train set)----')
print(lon,lat)

mtest = test.groupby(['lon','lat']).agency_name.agg(['count']).sort_values('count',ascending=False)
lon = mtest[mtest['count']>50].index.get_level_values(0).tolist()
lat = mtest[mtest['count']>50].index.get_level_values(1).tolist()
lon = round(sum(lon) / float(len(lon)),6)
lat = round(sum(lat) / float(len(lat)),6)
test.loc[test.lon.isna(),'lon'] = lon #fill in missing values
test.loc[test.lat.isna(),'lat'] = lat #fill in missing values
print(mtest[mtest['count']>50])
print('----locations replaced (test set)----')
print(lon,lat)



                      count   sum      mean       std
lon        lat                                       
-83.238943 42.377249    387  13.0  0.033592  0.180409
-83.262245 42.341729    355  42.0  0.118310  0.323431
-83.262271 42.341730    329  48.0  0.145897  0.353540
-83.046409 42.410855    264  12.0  0.045455  0.208695
-83.252023 42.352331    248  12.0  0.048387  0.215017
-83.277150 42.407284    233  15.0  0.064378  0.245953
-83.256895 42.349328    116   6.0  0.051724  0.222430
-83.136195 42.410644    107   5.0  0.046729  0.212051
-83.255176 42.348420     86   2.0  0.023256  0.151599
-83.257840 42.350713     86   4.0  0.046512  0.211825
-83.284184 42.428686     75  10.0  0.133333  0.342224
-83.045493 42.330486     52   0.0  0.000000  0.000000
----locations replaced (train set)----
-83.214569 42.370788
                      count
lon        lat             
-83.262271 42.341730    260
-83.262245 42.341729    225
-83.277150 42.407284    153
-83.252023 42.352331     58
-83.238943 42.37

In [162]:
#Variables to use in models
ycol = ['compliance']
xcol = ['state_cat','judgment_level','discount','disposition','agency_name','lat','lon']

X = train[xcol]
Xval = test[xcol]
y = np.ravel(train[ycol].astype('int'))

#check missing data
print('-----check missing data-----')
print('--train set--')
print(X.isna().sum())
print('--test set--')
print(Xval.isna().sum())

#dummy variables
X = pd.get_dummies(X)
Xval = pd.get_dummies(Xval)

-----check missing data-----
--train set--
state_cat         0
judgment_level    0
discount          0
disposition       0
agency_name       0
lat               0
lon               0
dtype: int64
--test set--
state_cat         0
judgment_level    0
discount          0
disposition       0
agency_name       0
lat               0
lon               0
dtype: int64


In [157]:
#Libraries to test data
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC

In [163]:
#split in two parameters
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

In [164]:
#dummy classifier
dummy_clf = DummyClassifier(strategy = 'most_frequent').fit(Xtrain,ytrain)
ytrpred = dummy_clf.predict(Xtrain)
ytepred = dummy_clf.predict(Xtest)
print('Train, test and AUCROC for dummy Classifier')
dummy_clf.score(Xtrain,ytrain), dummy_clf.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Train, test and AUCROC for dummy Classifier


(0.9274616584243051, 0.9274706029522142, 0.5, 0.5)

In [165]:
#Logistic regression
m1 = LogisticRegression(max_iter = 10000).fit(Xtrain,ytrain)
ytrpred = m1.predict(Xtrain)
ytepred = m1.predict(Xtest)
print('Train, test and AUCROC for logistic regression')
m1.score(Xtrain,ytrain), m1.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Train, test and AUCROC for logistic regression


(0.9366436214129048,
 0.9369276957718289,
 0.5671585736260306,
 0.5682156218525467)

In [167]:
#decision trees
grid_values = {'max_features': [1,2,3,4,5], 'max_depth': [4,6,8,10,15]} 
dtc = DecisionTreeClassifier()
grid_dtc = GridSearchCV(dtc, param_grid = grid_values, scoring='roc_auc')
grid_dtc.fit(Xtrain, ytrain)
print('Grid best parameter: ', grid_dtc.best_params_)
ytrpred = grid_dtc.predict(Xtrain)
ytepred = grid_dtc.predict(Xtest)
print('Train, test and AUCROC for Decision tree')
grid_dtc.score(Xtrain,ytrain), grid_dtc.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Grid best parameter:  {'max_depth': 10, 'max_features': 5}
Train, test and AUCROC for Decision tree


(0.7990625220178151,
 0.7858226081536629,
 0.5802791668078394,
 0.5745856689291738)

In [168]:
#random forests
#n_estimators, number of bootstrap samples
#max_depth, maximum depth of the tree
model = RandomForestClassifier(n_jobs=2, random_state=0)
grid_values = {'n_estimators':[5,10,20,25], 'max_depth':[5,8,10,15], 'max_features':[5,8,9,10]}
grid_model = GridSearchCV(model, param_grid=grid_values, scoring="roc_auc")
grid_model.fit(Xtrain, ytrain)
print('Grid best parameter: ', grid_model.best_params_)
ytrpred = grid_model.predict(Xtrain)
ytepred = grid_model.predict(Xtest)
print('Train, test and AUCROC for random forests')
grid_model.score(Xtrain,ytrain), grid_model.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Grid best parameter:  {'max_depth': 15, 'max_features': 10, 'n_estimators': 25}
Train, test and AUCROC for random forests


(0.9014534291340515,
 0.8102327885232656,
 0.6386207731762368,
 0.6007063499314763)

In [170]:
#neural network
scaler = MinMaxScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

nn = MLPClassifier(hidden_layer_sizes = [50], random_state = 0, solver='lbfgs', max_iter=10000)
grid_values = {'activation':['relu'], 'alpha':[0.01,1]}
grid_nn = GridSearchCV(nn, param_grid=grid_values, scoring="roc_auc")
grid_nn.fit(Xtrain_scaled, ytrain)
print('Grid best parameter: ', grid_nn.best_params_)
ytrpred = grid_nn.predict(Xtrain_scaled)
ytepred = grid_nn.predict(Xtest_scaled)
print('Train, test and AUCROC for neural networks')
grid_nn.score(Xtrain_scaled,ytrain), grid_nn.score(Xtest_scaled,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Grid best parameter:  {'activation': 'relu', 'alpha': 0.01}
Train, test and AUCROC for neural networks


(0.7915552855669578, 0.792140477309937, 0.5804773140930667, 0.5794688709225632)

In [172]:
from sklearn.ensemble import RandomForestRegressor
    
model = RandomForestRegressor(n_jobs=2, random_state=0)
grid_values = {'n_estimators':[5,10,20,25], 'max_depth':[5,8,10,15], 'max_features':[5,8,9,10]}
grid_res = GridSearchCV(model, param_grid=grid_values, scoring="roc_auc")
grid_res.fit(Xtrain, ytrain)
print('Grid best parameter: ', grid_res.best_params_)
print('Grid best score (AUC): ', grid_res.best_score_)
ytrpred = grid_res.predict(Xtrain)
ytepred = grid_res.predict(Xtest)
print('Train, test and AUCROC for random forests')
grid_res.score(Xtrain,ytrain), grid_res.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Grid best parameter:  {'max_depth': 15, 'max_features': 10, 'n_estimators': 25}
Grid best score (AUC):  0.808293342446263
Train, test and AUCROC for random forests


(0.900754172734513, 0.8094909687719776, 0.900754172734513, 0.8094909687719776)

In [6]:
#impossible to compute, takes a long time.

#grid_values = {'C': [0.1, 1], 'gamma': [0.1, 1.0]}, 'class_weight':[{1:10},{1:20},{1:50}]}
#scaler = StandardScaler()
#Xtrain_scaled = scaler.fit_transform(Xtrain)
#Xtest_scaled = scaler.transform(Xtest)

#svm = SVC(kernel='rbf', random_state =0)
#grid_svm_acc = GridSearchCV(svm, param_grid = grid_values, scoring = 'roc_auc')
#grid_svm_acc.fit(Xtrain_scaled, ytrain)
#y_decision_fn_scores_acc = grid_svm_acc.decision_function(Xtest_scaled) 

#print('Grid best parameter (max. accuracy): ', grid_svm_acc.best_params_)
#print('Grid best score (accuracy): ', grid_svm_acc.best_score_)

##Best SVM model 
##svm = SVC(random_state =0).fit(Xtrain_scaled,ytrain)
##ypred = svm.predict(Xtest_scaled)
##svm.score(Xtest_scaled,ytest), svm.score(Xtrain_scaled,ytrain), roc_auc_score(ytest, ypred)

In [ ]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return # Your answer here